RESPIRATORY SOUND DATABASE MODEL TRAINING

In [ ]:
# LIGHT GBM SAVED MODEL LINK -> https://drive.google.com/file/d/1g9EdkJIMYl7Ow4GdEKS6M3DGAVwHNj-O/view?usp=sharing

In [6]:
import pandas as pd
import os
import numpy as np
import IPython.display as ipd
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Attention
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
features_path = "/content/drive/MyDrive/respiratory_sound_db/respiratory_sound_db_database.csv"

In [7]:
extracted_features_df = pd.read_csv(features_path)

In [8]:
extracted_features_df.sample(5)

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,...,melspec6,melspec7,melspec8,melspec9,melspec10,zcr,sc,sr,chroma,severity
631,-294.78506,96.655106,34.659172,34.723965,19.786650,23.064861,14.580255,15.925095,7.676233,10.419303,...,0.001773,0.001057,0.000630,4.300131e-04,3.029853e-04,0.016008,1207.756225,2912.290781,0.878826,moderate
328,-199.29330,170.485140,51.252120,26.918268,15.217001,13.579975,2.502592,0.515945,-0.071960,7.946166,...,0.007463,0.002088,0.000980,6.050544e-04,4.076105e-04,0.011124,494.992802,592.000713,0.764454,moderate
580,-314.72873,102.887184,31.823938,24.601925,13.530396,21.941830,16.871117,18.184374,10.269935,11.879996,...,0.000710,0.000469,0.000359,2.428102e-04,1.851650e-04,0.012181,1102.749024,2382.703883,0.879624,moderate
206,-481.70870,120.617270,83.466210,50.906050,32.726000,24.247194,19.394196,15.823188,12.701819,9.509219,...,0.000007,0.000004,0.000003,1.636363e-06,1.078164e-06,0.004629,216.942981,179.310130,0.838975,severe
365,-509.68670,136.920730,75.090160,27.396720,12.164269,15.591138,18.815252,15.443967,10.063177,8.323277,...,0.000003,0.000002,0.000001,9.086825e-07,6.472325e-07,0.003666,237.840727,184.618489,0.852776,moderate


In [9]:
X = extracted_features_df.drop(["severity"], axis=1)
y = extracted_features_df["severity"]

In [10]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [11]:
df_train = X_train.assign(severity = y_train)
df_train.head()

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,...,melspec6,melspec7,melspec8,melspec9,melspec10,zcr,sc,sr,chroma,severity
378,-307.73600,125.243095,48.223534,38.952072,26.603790,30.847189,17.453247,20.060024,13.430505,11.745695,...,0.000675,3.404742e-04,2.505727e-04,2.576655e-04,1.240719e-04,0.008150,549.228690,451.510301,0.793106,moderate
356,-296.44614,93.332810,8.953807,41.458324,6.063129,23.101992,20.022984,3.880507,0.465956,16.388271,...,0.002243,1.089326e-02,1.435100e-02,3.873638e-03,1.587028e-03,0.038429,1645.694374,3947.520755,0.799098,moderate
738,-352.98260,119.739890,58.241062,45.309910,27.418768,22.756527,16.143890,15.447886,11.353783,11.095871,...,0.000240,1.651057e-04,1.198968e-04,8.864825e-05,6.748416e-05,0.005687,486.191717,326.645202,0.822959,moderate
85,-378.06012,89.016266,46.741920,18.745758,17.230076,25.899021,27.672373,22.983524,19.895712,19.288288,...,0.000074,7.944382e-07,2.320163e-07,9.343476e-08,5.001076e-08,0.001157,66.653894,81.977536,0.730808,moderate
427,-388.41560,104.036780,50.460167,44.625750,31.439820,28.702192,21.286316,19.661304,14.641257,13.728268,...,0.000166,1.142533e-04,8.278599e-05,6.083334e-05,4.563552e-05,0.004760,451.661059,212.696587,0.836391,moderate


In [12]:
df_test = X_test.assign(severity = y_test)
df_test.head()

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,...,melspec6,melspec7,melspec8,melspec9,melspec10,zcr,sc,sr,chroma,severity
306,-420.78915,78.128700,54.416286,38.684834,30.835783,26.365980,20.766314,14.727966,11.381536,10.897570,...,1.198652e-04,7.524193e-05,2.218794e-05,1.142393e-05,5.716828e-06,0.002228,233.764703,155.166463,0.803567,mild
711,-412.39725,65.986950,52.545364,35.822548,21.135767,11.782062,8.251705,8.979198,11.708162,14.550409,...,3.203884e-08,2.222905e-09,6.250429e-10,2.381319e-10,1.213777e-10,0.004820,98.637711,192.698835,0.756212,moderate
298,-313.22266,129.190020,44.217056,22.628693,27.262625,29.384330,25.945171,17.809134,11.249167,11.140881,...,6.022816e-04,2.503096e-04,1.496353e-04,1.058791e-04,7.800374e-05,0.005470,372.195864,323.585089,0.821056,moderate
466,-298.91925,128.519680,41.174675,31.972239,26.385944,31.515370,20.517624,18.974436,10.214501,10.581030,...,8.069946e-04,4.017600e-04,2.619916e-04,2.082494e-04,1.404153e-04,0.007177,585.102123,514.436216,0.803703,moderate
253,-476.93280,56.167046,53.156050,49.026382,43.795277,37.923744,31.872444,26.070625,20.846584,16.434586,...,2.927532e-06,1.610909e-06,9.354920e-07,5.823070e-07,3.931878e-07,0.001117,62.281930,48.712002,0.842608,severe


In [13]:
!pip install --pre pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 94.8 MB/s eta 0:0

In [14]:
from pycaret.classification import *

s = setup(df_train, target = 'severity')

,Description,Value
0,Session id,5061
1,Target,severity
2,Target type,Multiclass
3,Target mapping,"healthy: 0, mild: 1, moderate: 2, severe: 3"
4,Original data shape,"(736, 31)"
5,Transformed data shape,"(736, 31)"
6,Transformed train set shape,"(515, 31)"
7,Transformed test set shape,"(221, 31)"
8,Numeric features,30
9,Preprocess,True


In [15]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9010,0.9497,0.9010,0.8714,0.8828,0.4311,0.4574,1.0640
et,Extra Trees Classifier,0.8970,0.9688,0.8970,0.8305,0.8604,0.2728,0.3240,0.4970
xgboost,Extreme Gradient Boosting,0.8952,0.9567,0.8952,0.8623,0.8762,0.4072,0.4270,0.4630
gbc,Gradient Boosting Classifier,0.8913,0.9454,0.8913,0.8451,0.8657,0.3431,0.3663,2.6730
knn,K Neighbors Classifier,0.8874,0.8888,0.8874,0.8540,0.8678,0.3707,0.4013,0.2800
qda,Quadratic Discriminant Analysis,0.8816,0.5000,0.8816,0.7772,0.8261,0.0000,0.0000,0.1500
dummy,Dummy Classifier,0.8816,0.5000,0.8816,0.7772,0.8261,0.0000,0.0000,0.0780
rf,Random Forest Classifier,0.8815,0.9567,0.8815,0.8090,0.8418,0.1611,0.1977,0.8170
ridge,Ridge Classifier,0.8796,0.0000,0.8796,0.7785,0.8260,0.0090,0.0106,0.1220
lr,Logistic Regression,0.8681,0.9277,0.8681,0.8393,0.8518,0.2999,0.3107,1.4240


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [16]:
finalize_model(best)

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['mfcc1', 'mfcc2', 'mfcc3', 'mfcc4',
                                             'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8',
                                             'mfcc9', 'mfcc10', 'mfcc11',
                                             'mfcc12', 'mfcc13', 'mfcc14',
                                             'mfcc15', 'mf...
                 LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_leaves=31, objective=None,
                                random_state=5061, reg_alpha=0.0,
                                reg_lambda=0.0, silent='warn', subsample=1.0,
                                subsample_for_bin=200000, subsample_freq=0))],
         verbose=False)

In [17]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [18]:
predict_model(best, df_test)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9022,0.9866,0.9022,0.8735,0.8807,0.4672,0.4833


,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,...,melspec8,melspec9,melspec10,zcr,sc,sr,chroma,severity,prediction_label,prediction_score
306,-420.789154,78.128700,54.416286,38.684834,30.835783,26.365980,20.766314,14.727966,11.381536,10.897570,...,2.218793e-05,1.142393e-05,5.716828e-06,0.002228,233.764709,155.166458,0.803567,1,moderate,0.9763
711,-412.397247,65.986954,52.545364,35.822548,21.135767,11.782062,8.251705,8.979198,11.708162,14.550409,...,6.250429e-10,2.381319e-10,1.213777e-10,0.004820,98.637711,192.698837,0.756212,2,moderate,1.0000
298,-313.222656,129.190018,44.217056,22.628693,27.262625,29.384331,25.945171,17.809134,11.249167,11.140881,...,1.496353e-04,1.058791e-04,7.800374e-05,0.005470,372.195862,323.585083,0.821056,2,moderate,1.0000
466,-298.919250,128.519684,41.174675,31.972239,26.385944,31.515369,20.517624,18.974436,10.214501,10.581030,...,2.619915e-04,2.082494e-04,1.404153e-04,0.007177,585.102112,514.436218,0.803703,2,moderate,1.0000
253,-476.932800,56.167046,53.156052,49.026382,43.795277,37.923744,31.872444,26.070625,20.846584,16.434586,...,9.354920e-07,5.823070e-07,3.931878e-07,0.001117,62.281929,48.712002,0.842608,3,moderate,0.9871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,-397.792603,75.166420,41.437260,21.385164,20.664238,28.250284,31.300079,25.563833,18.093590,15.354956,...,3.943183e-05,2.781670e-06,5.107695e-07,0.000958,89.365555,58.659157,0.842015,2,moderate,0.9998
293,-307.692963,131.974747,45.330147,22.170235,28.774511,31.557947,26.961538,16.899775,10.246912,11.488774,...,1.504142e-04,1.009065e-04,7.414753e-05,0.006064,371.011383,346.729523,0.794913,2,moderate,1.0000
761,-526.163330,119.899986,66.829025,25.123028,14.423087,21.830912,25.023727,17.701427,9.267004,7.407423,...,4.024330e-07,3.458606e-07,3.131529e-07,0.008640,409.587402,400.162842,0.776355,2,moderate,1.0000
2,-596.806091,116.009735,60.592537,25.570189,26.233389,38.501705,36.034607,18.649155,3.548372,0.805531,...,1.557648e-07,1.265123e-07,1.069120e-07,0.002278,192.965317,138.804230,0.816630,0,moderate,0.9604


In [19]:
!pip install lightgbm

In [21]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.model_selection import KFold

In [33]:
# Assume you have your feature matrix X and target vector y
X = X
y = y

In [34]:
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [40]:
import lightgbm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import numpy as np

# Define the model
lgbmclf = lightgbm.LGBMClassifier(random_state=1574)
# Create a cross-validation object (in this example, using 5-fold cross-validation)
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Perform cross-validation
acc_scores = cross_val_score(lgbmclf , X_train, y_train, cv=kf, scoring='accuracy')
# Print the cross-validation scores
print("Cross-validation Accuracy scores:", acc_scores)
# Print the average cross-validation score
print("Average Cross Validation Accuracy score:", np.mean(acc_scores))


# Perform cross-validation
f1_weighted_scores = cross_val_score(lgbmclf , X_train, y_train, cv=kf, scoring='f1_weighted')
# Print the cross-validation scores
print("Cross-validation F1 scores:", f1_weighted_scores)
# Print the average cross-validation score
print("Average Cross Validation F1 score:", np.mean(f1_weighted_scores))

Cross-validation Accuracy scores: [0.95945946 0.90540541 0.86486486 0.90540541 0.94594595 0.90540541
 0.93150685 0.89041096 0.95890411 0.91780822]
Average Cross Validation Accuracy score: 0.9185116623472789
Cross-validation F1 scores: [0.94884738 0.89311602 0.8621982  0.87694943 0.93230373 0.89613213
 0.92099361 0.86211089 0.97079569 0.9156844 ]
Average Cross Validation F1 score: 0.9079131480160532


In [41]:
#from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import accuracy_score,f1_score
lgbmclf = lightgbm.LGBMClassifier(random_state=1574)
lgbmclf.fit(X_train, y_train)
y_test_pred = lgbmclf.predict(X_test)

avg_accuracy_score = accuracy_score(y_test, y_test_pred)
print(f"Average Test Accuracy Score:  {avg_accuracy_score}")

avg_f1_score = f1_score(y_test, y_test_pred, average="weighted")
print(f"Average Test F1 Score:  {avg_f1_score}")

Average Test Accuracy Score:  0.9130434782608695
Average Test F1 Score:  0.900689165906557


In [42]:
import joblib
joblib.dump(lgbmclf, 'respiratory_db_lightgbm_model.joblib')

['respiratory_db_lightgbm_model.joblib']